In [25]:
# Step 0: Install required libraries (uncomment when running locally)
#%pip install pymupdf pdfplumber pytesseract faiss-cpu sentence-transformers requests


In [26]:
# %pip install "numpy<2.0"

In [27]:
import os
import fitz  
import pdfplumber
import pytesseract
from PIL import Image
from io import BytesIO
import torch
from sentence_transformers import SentenceTransformer, util
import faiss
import requests
import pandas as pd
from PIL import ImageEnhance, ImageFilter
import pandas as pd
from datetime import datetime
import os

In [28]:
# --------- STEP 1: Extract content from PDF ---------
def extract_pdf_content(pdf_path):
    text_blocks = []
    tables = []
    images = []

    with pdfplumber.open(pdf_path) as pdf:
        for i, page in enumerate(pdf.pages):
            text = page.extract_text()
            if text:
                text_blocks.append({"page": i + 1, "text": text})

            
            page_tables = page.extract_tables()
            for tbl in page_tables:
                tables.append({"page": i + 1, "table": tbl})

    
    pdf_doc = fitz.open(pdf_path)
    for page_num in range(len(pdf_doc)):
        page = pdf_doc[page_num]
        image_list = page.get_images(full=True)
        for img_index, img in enumerate(image_list):
            xref = img[0]
            base_image = pdf_doc.extract_image(xref)
            img_bytes = base_image["image"]
            image = Image.open(BytesIO(img_bytes)).convert("RGB")
            images.append({"page": page_num + 1, "image": image})

    return text_blocks, tables, images


In [29]:
# --------- STEP 2: OCR images ---------


def preprocess_image_for_ocr(img: Image.Image) -> Image.Image:
    
    img = img.convert("L")
    
    img = ImageEnhance.Contrast(img).enhance(2.0)
    
    img = img.resize((img.width * 2, img.height * 2), Image.LANCZOS)
    # Optional: apply slight sharpening or denoise
    img = img.filter(ImageFilter.SHARPEN)
    return img

def ocr_images(images):
    img_text_blocks = []
    for img_obj in images:
        processed_img = preprocess_image_for_ocr(img_obj["image"])
        text = pytesseract.image_to_string(processed_img)
        if text.strip():
            img_text_blocks.append({
                "page": img_obj["page"],
                "text": f"[OCR from Image Page {img_obj['page']}]\n{text.strip()}"
            })
    return img_text_blocks


In [30]:
# --------- STEP 3: Local LLM Answering via Ollama HTTP API ---------
def query_ollama_http(question, context):
    prompt = f"""You are a helpful and intelligent and smart assistant. Use the following context to answer the question.

Context:
{context}

Question:
{question}

Answer:"""

    response = requests.post(
        "http://localhost:11434/api/generate",
        json={
            "model": "mistral",
            "prompt": prompt,
            "stream": False
        }
    )

    return response.json().get("response", "").strip()

In [31]:
# --------- STEP 4: Embed and search ---------
class PDFIndex:
    def __init__(self):
        self.model = SentenceTransformer('all-MiniLM-L6-v2')
        self.texts = []
        self.embeddings = []
        self.index = None

    def add_documents(self, docs):
        self.texts = [doc["text"] for doc in docs]
        self.embeddings = self.model.encode(self.texts, convert_to_tensor=True)
        embeddings_np = self.embeddings.cpu().detach().numpy().astype('float32')
        dim = embeddings_np.shape[1]
        self.index = faiss.IndexFlatL2(dim)
        self.index.add(embeddings_np)

    def retrieve_context(self, question, top_k=4):
        q_embedding = self.model.encode([question], convert_to_tensor=True).cpu().detach().numpy().astype('float32')
        scores, indices = self.index.search(q_embedding, top_k)
        return "\n\n".join([self.texts[i] for i in indices[0]])

In [32]:
# --------- STEP 5: Run all steps ---------
def run_pipeline(pdf_path):
    print("Extracting from PDF... ⏳")
    text_blocks, tables, images = extract_pdf_content(pdf_path)

    print("Running OCR on images... 🧠")
    ocr_blocks = ocr_images(images)

    all_blocks = text_blocks + ocr_blocks

    print("Indexing documents... 📚")
    index = PDFIndex()
    index.add_documents(all_blocks)

    print("System ready for QA. ✅")
    return index, tables


In [33]:


def format_log_entry(serial, timestamp, question, answer):
    return (
        f"{serial}. {timestamp}\n"
        f"Question: {question}\n"
        f"Answer: {answer}\n\n"
    )

def run_qa_interface(pdf_path):
    index, tables = run_pipeline(pdf_path)

    base_name = os.path.splitext(os.path.basename(pdf_path))[0]
    log_file = f"{base_name}_qa_log.txt"  # grouped by PDF name

    
    serial = 1
    if os.path.exists(log_file):
        with open(log_file, "r", encoding="utf-8") as f:
            content = f.read()
            serial = content.count("Question:") + 1

    print("PDF loaded. Ask your questions below. Type 'exit' to quit.\n")

    while True:
        question = input("❓ Your question: ")
        if question.strip().lower() == "exit":
            print("Exiting. Goodbye!")
            break

        context = index.retrieve_context(question)
        # print(context)
        answer = query_ollama_http(question, context)
        print(f"\n💬 Answer: {answer}\n")

        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        formatted = format_log_entry(serial, timestamp, question, answer)

        with open(log_file, "a", encoding="utf-8") as f:
            f.write(formatted + "\n")  # 2-line gap

        serial += 1


In [34]:


# LOG_FILE = "qa_log.csv"

# def run_qa_interface(pdf_path):
#     index, tables = run_pipeline(pdf_path)

#     print("PDF loaded. Ask your questions below. Type 'exit' to quit.\n")

#     while True:
#         question = input("❓ Your question: ")
#         if question.strip().lower() == "exit":
#             print("Exiting. Goodbye!")
#             break

#         context = index.retrieve_context(question)
#         answer = query_ollama_http(question, context)
#         print(f"\n💬 Answer: {answer}\n")

#         # Save with timestamp
#         entry = {
#             "Timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
#             "Question": question,
#             "Answer": answer
#         }

#         # Append to CSV (preserve previous log)
#         if os.path.exists(LOG_FILE):
#             existing_df = pd.read_csv(LOG_FILE)
#             updated_df = pd.concat([existing_df, pd.DataFrame([entry])], ignore_index=True)
#         else:
#             updated_df = pd.DataFrame([entry])

#         updated_df.to_csv(LOG_FILE, index=False)

def summarize_pdf(pdf_path):
    index, _ = run_pipeline(pdf_path)
    print("📚 Generating document summary...\n")
    summary_context = index.retrieve_context("summarize this document")
    summary = query_ollama_http("Summarize this document", summary_context)
    print("📄 Summary:\n", summary)
    return summary


In [35]:
# %pip install --upgrade numpy

In [36]:
# Example usage:
run_qa_interface("./sample.pdf")

CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, def

Extracting from PDF... ⏳


CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, def

Running OCR on images... 🧠
Indexing documents... 📚
System ready for QA. ✅
PDF loaded. Ask your questions below. Type 'exit' to quit.


💬 Answer: The document does not provide specific information about the HRA for executives in Grade E-3 for a Class X city. However, it states that the rates of HRA for employees in IDA pay pattern as of 15.05.2018 are 24% of Basic Pay for Class X cities (X-Class which has a population of 50 Lakh and above). Therefore, to find the exact amount of HRA for an executive in Grade E-3, you would need to know their basic pay.


💬 Answer: To find the House Rent Allowance (HRA) for executives in Grade E-3 for a Class X city, we need to consider the revised rates mentioned in the context.

A Class X city has a population of 50 Lakh and above, and the initial HRA rate is 24% of Basic Pay. However, when IDA (Industrial Dearness Allowance) crosses 25%, the HRA rate gets revised to 27%. Since we don't have the exact Basic Pay for an Executive in Grade E-3 or the curr

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

In [ ]:
# summarize_pdf("./sample.pdf")
